##### Copyright 2019 The TensorFlow Authors.



In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Load CSV with tf.data
# tf.data を使って CSV をロードする

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/alpha/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/じja/r2/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ja/r2/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This tutorial provides an example of how to load CSV data from a file into a `tf.data.Dataset`.

このチュートリアルでは、CSV データをどうやってファイルから `tf.data.Dataset` にロードするかの例を示します。

The data used in this tutorial are taken from the Titanic passenger list. We'll try to predict the likelihood a passenger survived based on characteristics like age, gender, ticket class, and whether the person was traveling alone.

このチュートリアルで使われているデータはタイタニック号の乗客リストから取られたものです。乗客が生き残る可能性を、年齢、性別、チケットの等級、そして乗客が一人で旅行しているか否かといった特性から予測することを試みます。

## Setup
## 設定

In [1]:
!pip install tensorflow==2.0.0-alpha0

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [28]:
!pip install tfds-nightly

    100% |████████████████████████████████| 839kB 10.9MB/s 
    100% |████████████████████████████████| 153kB 13.5MB/s 
    100% |████████████████████████████████| 440kB 15.3MB/s 
  Stored in directory: /Users/masatoshi/Library/Caches/pip/wheels/5b/d7/0f/e58eae695403de585269f4e4a94e0cd6ca60ec0c202936fa4a
  Stored in directory: /Users/masatoshi/Library/Caches/pip/wheels/17/08/ec/22b464874958c3fc91e1a75748fae2220eb704a8b1035f9a03
Successfully built dill psutil
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [3]:
# Make numpy values easier to read.
# numpy の値を読みやすくする
np.set_printoptions(precision=3, suppress=True)

## Load data
## データのロード

So we know what we're doing, lets look at the top of the CSV file we're working with.

何をしているかはわかっていますが、扱っている CSV ファイルの先頭を見てみましょう。

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


As you can see, the columns in the CSV are labeled. We need the list later on, so let's read it out of the file.

ご覧のように、CSV の列にはラベルが付いています。後ほど必要になるので、ファイルから読み出しておきましょう。

In [5]:
# CSV columns in the input file.
# 入力ファイル中の CSV 列
with open(train_file_path, 'r') as f:
    names_row = f.readline()


CSV_COLUMNS = names_row.rstrip('\n').split(',')
print(CSV_COLUMNS)

['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']


 The dataset constructor will pick these labels up automatically.
 
 データセットコンストラクタはこれらのラベルを自動的にピックアップします。

If the file you are working with does not contain the column names in the first line, pass them in a list of strings to  the `column_names` argument in the `make_csv_dataset` function.

使用するファイルの1行目にコラム名がない場合、`make_csv_dataset` 関数の `column_names` 引数に文字列のリストとして渡します。

```python

CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

dataset = tf.data.experimental.make_csv_dataset(
     ...,
     column_names=CSV_COLUMNS,
     ...)
  
```


This example is going to use all the available columns. If you need to omit some columns from the dataset, create a list of just the columns you plan to use, and pass it into the (optional) `select_columns` argument of the constructor.

この例では使用可能な列をすべて使うことになります。データセットから列を除く必要がある場合には、使用したい列だけを含むリストを作り、コンストラクタの（オプションである）`select_columns` 引数として渡します。


```python

drop_columns = ['fare', 'embark_town']
columns_to_use = [col for col in CSV_COLUMNS if col not in drop_columns]

dataset = tf.data.experimental.make_csv_dataset(
  ...,
  select_columns = columns_to_use, 
  ...)

```

We also have to identify which column will serve as the labels for each example, and what those labels are.

各サンプルのラベルとなる列を特定し、それが何であるかを示す必要があります。

In [6]:
LABELS = [0, 1]
LABEL_COLUMN = 'survived'

FEATURE_COLUMNS = [column for column in CSV_COLUMNS if column != LABEL_COLUMN]

Now that these constructor argument values are in place,  read the CSV data from the file and create a dataset. 

コンストラクタの引数の値が揃ったので、ファイルから CSV データを読み込みデータセットを作ることにしましょう。

(For the full documentation, see `tf.data.experimental.make_csv_dataset`)

（完全なドキュメントは、`tf.data.experimental.make_csv_dataset` を参照してください）

In [7]:
def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Each item in the dataset is a batch, represented as a tuple of (*many examples*, *many labels*). The data from the examples is organized in column-based tensors (rather than row-based tensors), each with as many elements as the batch size (12 in this case).

データセットを構成する要素は、(複数のサンプル, 複数のラベル)の形のタプルとして表されるバッチです。サンプル中のデータは（行ベースのテンソルではなく）列ベースのテンソルとして構成され、それぞれはバッチサイズ（このケースでは12個）の要素が含まれます。

It might help to see this yourself.

実際に見てみましょう。

In [8]:
examples, labels = next(iter(raw_train_data)) # Just the first batch.
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: id=169, shape=(12,), dtype=string, numpy=
array([b'female', b'female', b'male', b'male', b'male', b'female',
       b'male', b'female', b'male', b'male', b'male', b'female'],
      dtype=object)>), ('age', <tf.Tensor: id=161, shape=(12,), dtype=float32, numpy=
array([15. , 34. , 21. , 18. , 55.5, 31. , 18. , 33. , 36. , 40. , 47. ,
       25. ], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: id=167, shape=(12,), dtype=int32, numpy=array([0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1], dtype=int32)>), ('parch', <tf.Tensor: id=168, shape=(12,), dtype=int32, numpy=array([0, 1, 0, 0, 0, 0, 0, 2, 0, 4, 0, 1], dtype=int32)>), ('fare', <tf.Tensor: id=166, shape=(12,), dtype=float32, numpy=
array([  7.225,  32.5  ,   8.663,  73.5  ,   8.05 , 113.275,   7.75 ,
        27.75 ,  78.85 ,  27.9  ,   9.   ,  30.   ], dtype=float32)>), ('class', <tf.Tensor: id=163, shape=(12,), dtype=string, numpy=
array([b'Third', b'Second', b'Third', b'Second', b'Third', b'Fi

## Data preprocessing
## データ処理

### Categorical data
### カテゴリデータ

Some of the columns in the CSV data are categorical columns. That is, the content should be one of a limited set of options.

この CSV データ中のいくつかの列はカテゴリ列です。つまり、その中身は、限られた選択肢の中のひとつである必要があります。

In the CSV, these options are represented as text. This text needs to be converted to numbers before the model can be trained. To facilitate that, we need to create a list of categorical columns, along with a list of the options available in each column.

この CSV では、これらの選択肢はテキストとして表現されています。このテキストは、モデルの訓練を行えるように、数字に変換する必要があります。これをやりやすくするため、カテゴリ列のリストとその選択肢のリストを作成する必要があります。

In [9]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


Write a function that takes a tensor of categorical values, matches it to a list of value names, and then performs a one-hot encoding.

カテゴリ値のテンソルを受け取り、それを値の名前のリストとマッチングして、さらにワンホット・エンコーディングを行う関数を書きます。

In [10]:
def process_categorical_data(data, categories):
  """Returns a one-hot encoded tensor representing categorical values."""
  """カテゴリ値を表すワンホット・エンコーディングされたテンソルを返す"""
  
  # Remove leading ' '.
  # 最初の ' ' を取り除く
  data = tf.strings.regex_replace(data, '^ ', '')
  # Remove trailing '.'.
  # 最後の '.' を取り除く
  data = tf.strings.regex_replace(data, r'\.$', '')
  
  # ONE HOT ENCODE
  # ワンホット・エンコーディング
  # Reshape data from 1d (a list) to a 2d (a list of one-element lists)
  # data を1次元（リスト）から2次元（要素が1個のリストのリスト）にリシェープ
  data = tf.reshape(data, [-1, 1])
  # For each element, create a new list of boolean values the length of categories,
  # where the truth value is element == category label
  # それぞれの要素について、カテゴリ数の長さの真偽値のリストで、
  # 真は要素がカテゴリラベルに一致していることを示すを作成
  data = tf.equal(categories, data)
  # Cast booleans to floats.
  # 真偽値を浮動小数点数にキャスト
  data = tf.cast(data, tf.float32)
  
  # The entire encoding can fit on one line:
  # data = tf.cast(tf.equal(categories, tf.reshape(data, [-1, 1])), tf.float32)
  # エンコーディング全体を次の1行に収めることもできる：
  # data = tf.cast(tf.equal(categories, tf.reshape(data, [-1, 1])), tf.float32)
  return data

To help you visualize this, we'll take a single category-column tensor from the first batch, preprocess it, and show the before and after state.

これを可視化するため、最初のバッチからカテゴリ列のレンソル1つを取り出し、処理を行い、前後の状態を示します。

In [11]:
class_tensor = examples['class']
class_tensor

<tf.Tensor: id=163, shape=(12,), dtype=string, numpy=
array([b'Third', b'Second', b'Third', b'Second', b'Third', b'First',
       b'Third', b'Second', b'First', b'Third', b'Third', b'Second'],
      dtype=object)>

In [12]:
class_categories = CATEGORIES['class']
class_categories

['First', 'Second', 'Third']

In [13]:
processed_class = process_categorical_data(class_tensor, class_categories)
processed_class

<tf.Tensor: id=188, shape=(12, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)>

Notice the relationship between the lengths of the two inputs and the shape of the output.

2つの入力の長さと、出力の形状の関係に注目してください。

In [14]:
print("Size of batch: ", len(class_tensor.numpy()))
print("Number of category labels: ", len(class_categories))
print("Shape of one-hot encoded tensor: ", processed_class.shape)

Size of batch:  12
Number of category labels:  3
Shape of one-hot encoded tensor:  (12, 3)


### Continuous data
### 連続データ

Continuous data needs to be normalized, so that the values fall between 0 and 1. To do that, write a function that multiplies each value by 1 over twice the mean of the column values.

連続データは値が0と1の間にになるように標準化する必要があります。これを行うために、それぞれの値を、1を列値の平均の2倍で割ったものをかける関数を書きます。

The function should also reshape the data into a two dimensional tensor.

この関数は、データの2次元のテンソルへのリシェープも行います。

In [15]:
def process_continuous_data(data, mean):
  # Normalize data
  # data の標準化
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])

To do this calculation, you need the column means. You would obviously need to compute these in real life, but for this example we'll just provide them.

この計算を行うためには、列値の平均が必要です。現実には、この値を計算する必要があるのは明らかですが、この例のために値を示します。

In [16]:
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

Again, to see what this function is actually doing, we'll take a single tensor of continuous data and show it before and after processing.

前と同様に、この関数が実際に何をしているかを見るため、連続値のテンソルを1つ取り、処理前と処理後を見てみます。

In [17]:
age_tensor = examples['age']
age_tensor

<tf.Tensor: id=161, shape=(12,), dtype=float32, numpy=
array([15. , 34. , 21. , 18. , 55.5, 31. , 18. , 33. , 36. , 40. , 47. ,
       25. ], dtype=float32)>

In [18]:
process_continuous_data(age_tensor, MEANS['age'])

<tf.Tensor: id=197, shape=(12, 1), dtype=float32, numpy=
array([[0.253],
       [0.574],
       [0.354],
       [0.304],
       [0.937],
       [0.523],
       [0.304],
       [0.557],
       [0.607],
       [0.675],
       [0.793],
       [0.422]], dtype=float32)>

### Preprocess the data
### データの前処理

Now assemble these preprocessing tasks into a single function that can be mapped to each batch in the dataset. 

これらの前処理のタスクを1つの関数にまとめ、データセット内のバッチにマッピングできるようにします。

In [19]:
def preprocess(features, labels):
  
  # Process categorial features.
  # カテゴリ特徴量の処理
  for feature in CATEGORIES.keys():
    features[feature] = process_categorical_data(features[feature],
                                                 CATEGORIES[feature])

  # Process continuous features.
  # 連続特徴量の処理
  for feature in MEANS.keys():
    features[feature] = process_continuous_data(features[feature],
                                                MEANS[feature])
  
  # Assemble features into a single tensor.
  # 特徴量を1つのテンソルに組み立てる
  features = tf.concat([features[column] for column in FEATURE_COLUMNS], 1)
  
  return features, labels



Now apply that function with `tf.Dataset.map`, and shuffle the dataset to avoid overfitting.

次に、 `tf.Dataset.map` 関数を使って適用し、過学習を防ぐためにデータセットをシャッフルします。

In [20]:
train_data = raw_train_data.map(preprocess).shuffle(500)
test_data = raw_test_data.map(preprocess)

And let's see what a single example looks like.

サンプル1個がどうなっているか見てみましょう。

In [21]:
examples, labels = next(iter(train_data))

examples, labels

(<tf.Tensor: id=363, shape=(12, 24), dtype=float32, numpy=
 array([[0.   , 1.   , 0.152, 1.833, 2.634, 0.5  , 0.   , 0.   , 1.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 1.   ],
        [1.   , 0.   , 0.607, 0.   , 0.   , 0.153, 0.   , 1.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 1.   , 0.   ],
        [0.   , 1.   , 0.371, 0.   , 0.   , 0.153, 0.   , 0.   , 1.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 1.   , 0.   ],
        [1.   , 0.   , 0.456, 0.   , 0.   , 1.116, 1.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 1.   , 0.   , 0.   , 1.   , 0.   ],
        [0.   , 1.   , 0.81 , 0.917, 0.   , 0.576, 1.   , 0.   , 0.   ,
         1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.  

The examples are in a  two dimensional arrays of 12 items each (the batch size). Each item represents a single row in the original CSV file. The labels are a 1d tensor of 12 values.

このサンプルは、（バッチサイズである）12個のアイテムを持つ2次元の配列からできています。アイテムそれぞれは、元の CSV ファイルの1行を表しています。ラベルは12個の値を持つ1次元のテンソルです。

## Build the model
## モデルの構築

This example uses the [Keras Functional API](https://www.tensorflow.org/alpha/guide/keras/functional) wrapped in a `get_model` constructor to build up a simple model. 

この例では、[Keras Functional API](https://www.tensorflow.org/alpha/guide/keras/functional) を使用し、単純なモデルを構築するために `get_model` コンストラクタでラッピングしています。

In [22]:
def get_model(input_dim, hidden_units=[100]):
  """Create a Keras model with layers.
     複数の層を持つ Keras モデルを作成

  Args:
    input_dim: (int) The shape of an item in a batch. 
    labels_dim: (int) The shape of a label.
    hidden_units: [int] the layer sizes of the DNN (input layer first)
    learning_rate: (float) the learning rate for the optimizer.

  引数:
    input_dim: (int) バッチ中のアイテムの形状
    labels_dim: (int) ラベルの形状
    hidden_units: [int] DNN の層のサイズ（入力層が先）
    learning_rate: (float) オプティマイザの学習率
    
  Returns:
    A Keras model.
  
  戻り値:
    Keras モデル
  """

  inputs = tf.keras.Input(shape=(input_dim,))
  x = inputs

  for units in hidden_units:
    x = tf.keras.layers.Dense(units, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(inputs, outputs)
 
  return model

The `get_model` constructor needs to know the input shape of your data (not including the batch size).

`get_model` コンストラクタは入力データの形状（バッチサイズを除く）を知っている必要があります。

In [23]:
input_shape, output_shape = train_data.output_shapes

input_dimension = input_shape.dims[1] # [0] is the batch size

## Train, evaluate, and predict
## 訓練、評価、そして予測

Now the model can be instantiated and trained.

これでモデルをインスタンス化し、訓練することができます。

In [24]:
model = get_model(input_dimension)
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.fit(train_data, epochs=20)

Epoch 1/20
53/53 [==============================] - 1s 23ms/step - loss: 0.5913 - accuracy: 0.6922
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4958 - accuracy: 0.7895
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4570 - accuracy: 0.7943
Epoch 4/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4396 - accuracy: 0.8086
Epoch 5/20
53/53 [==============================] - 0s 4ms/step - loss: 0.4314 - accuracy: 0.8150
Epoch 6/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4264 - accuracy: 0.8150
Epoch 7/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4225 - accuracy: 0.8150
Epoch 8/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4190 - accuracy: 0.8198
Epoch 9/20
53/53 [==============================] - 0s 4ms/step - loss: 0.4159 - accuracy: 0.8214
Epoch 10/20
53/53 [==============================] - 0s 5ms/step - loss: 0.4129 - accuracy: 0.8246
Epoch 11/20
53/53 

Once the model is trained, we can check its accuracy on the `test_data` set.

モデルの訓練が終わったら、`test_data` データセットでの正解率をチェックできます。

In [25]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     22/Unknown - 0s 11ms/step - loss: 0.4420 - accuracy: 0.7992

Test Loss 0.44200402430512686, Test Accuracy 0.7992424368858337


Use `tf.keras.Model.predict` to infer labels on a batch or a dataset of batches.

単一のバッチは、バッチからなるデータセットのラベルを推論する場合には、`tf.keras.Model.predict` を使います。

In [27]:
predictions = model.predict(test_data)

# Show some results
# 結果のいくつかを表示
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))



Predicted survival: 11.02%  | Actual outcome:  DIED
Predicted survival: 87.44%  | Actual outcome:  SURVIVED
Predicted survival: 53.22%  | Actual outcome:  DIED
Predicted survival: 8.93%  | Actual outcome:  DIED
Predicted survival: 11.14%  | Actual outcome:  DIED
Predicted survival: 32.65%  | Actual outcome:  SURVIVED
Predicted survival: 9.22%  | Actual outcome:  DIED
Predicted survival: 78.39%  | Actual outcome:  SURVIVED
Predicted survival: 46.98%  | Actual outcome:  DIED
Predicted survival: 46.34%  | Actual outcome:  DIED
